<a href="https://colab.research.google.com/github/jett220201/Usable-and-No-Usable-Waste-Classifier-with-DL/blob/main/Models/MobileNetV3_Fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib notebook
#Only activate if you're using Jupyter Notebook

In [ ]:
import os
import cv2
import glob
import random
import sklearn
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from keras.models import Model, Sequential
from matplotlib.collections import EventCollection
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Read input images and assign labels based on folder names
print(os.listdir("data/"))

Control parameters

In [ ]:
Seed_0 = random.randint(1,2**32 - 1)
N_epochs = 50
Batch_size = 64
LR = 0.001
SIZE = 224  #Size in pixels 
Shuffle = True

In [ ]:
#Capture training data and labels into respective lists
images = []
labels = [] 

for directory_path in glob.glob("data/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        images.append(img)
        labels.append(label)

#Convert lists to arrays        
images = np.array(images)
labels = np.array(labels)

In [ ]:
#Spliting Dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state = Seed_0)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_test)
test_labels_encoded = le.transform(y_test)
le.fit(y_train)
train_labels_encoded = le.transform(y_train)

#print(test_labels_encoded)
print(train_labels_encoded)
len(train_labels_encoded)

#0 = Aprovechable
#1 = No_Aprovechable

# Preprocessing is needed

Note: each Keras Application expects a specific kind of input preprocessing. For MobileNetV3, by default input preprocessing is included as a part of the model (as a Rescaling layer), and thus `tf.keras.applications.mobilenet_v3.preprocess_input` is actually a pass-through function. In this use case, MobileNetV3 models expect their inputs to be float tensors of pixels with values in the [0-255] range. At the same time, preprocessing as a part of the model (i.e. Rescaling layer) can be disabled by setting include_preprocessing argument to False. With preprocessing disabled MobileNetV3 models expect their inputs to be float tensors of pixels with values in the [-1, 1] range.

Source: https://keras.io/api/applications/mobilenet/

In [ ]:
#Inputs
X_train = tf.keras.applications.mobilenet_v3.preprocess_input(X_train)
Y_train = train_labels_encoded
X_test = tf.keras.applications.mobilenet_v3.preprocess_input(X_test)
Y_test = test_labels_encoded

In [ ]:
#Create model MobileNetV3 for Fine Tunning
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top = True,
    weights="imagenet",
    pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing = True,
)

x = base_model.layers[-4].output
x = tf.keras.layers.Conv2D(2,(1,1), name = 'Logits')( x )
x = tf.keras.layers.BatchNormalization()( x )
x = tf.keras.layers.Flatten()( x )
outputs = tf.keras.layers.Dense(1, activation='sigmoid')( x )
model = tf.keras.models.Model( base_model.input , outputs )

for layer in model.layers[ : -10]:
    layer.trainable = False

model.summary()

In [ ]:
# Generating path to save weights per epoch 
checkpoint_path = "training_1/cp-{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path, 
    verbose = 1, 
    save_weights_only = True,
    save_freq = 'epoch')

# Model training

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate = LR  ),
              loss='binary_crossentropy',
              metrics=['accuracy',
                       'AUC',
                       tf.keras.metrics.TruePositives(),
                       tf.keras.metrics.TrueNegatives(),
                       tf.keras.metrics.FalsePositives(),
                       tf.keras.metrics.FalseNegatives()])

history = model.fit(x = X_train,
                    y = Y_train,
                    batch_size = Batch_size,
                    epochs = N_epochs,
                    verbose = 1,
                    shuffle = Shuffle,
                    validation_data = (X_test, Y_test),
                    callbacks=[cp_callback])


In [ ]:
# Get values of TN, FP, FN, TP for the confusion matrix

FN = history.history['val_false_negatives_2'] # For each compile the number plus by 
TN = history.history['val_true_negatives_2']  # 1 -> false_negatives_1,2,3....
FP = history.history['val_false_positives_2']
TP = history.history['val_true_positives_2']

In [ ]:
#Calculate F1 Score - Validation
F1 = []
for i in range(N_epochs):
    try:
        temp = ( ( 2*TP[i] ) / ( 2*TP[i] + FP[i] + FN[i] ) )
    except:
        temp = "Error"
    F1.append(temp)
print('-Result for F1 Score-')
print(F1,'\n')


#Calculate MCC - Validation
MCC = []
for i in range(N_epochs):
    try:
        temp = ( (TP[i]*TN[i]) - (FP[i]*FN[i]) ) / ( np.sqrt( (TP[i]+FP[i])*(TP[i]+FN[i])*(TN[i]+FP[i])*(TN[i]+FN[i]) ) )
    except:
        temp = "Error"
    MCC.append(temp)
print('-Result for MCC-')
print(MCC, '\n')

#Calculate Recall - Validation
Recall = []
for i in range(N_epochs):
    try:
        temp = ( TP[i] / ( TP[i] + FN[i] ) )
    except:
        temp = "Error"
    Recall.append(temp)
print('-Result for Recall-')
print(Recall, '\n')

#Calculate Specificity - Validation
Specificity = []
for i in range(N_epochs):
    try:
        temp = ( TN[i] / ( TN[i] + FP[i] )  )
    except:
        temp = "Error"
    Specificity.append(temp)
print('-Result for Specificity-')
print(Specificity, '\n')

In [ ]:
#Show Validation metrics
xdata = np.linspace(1, N_epochs + 1, num = N_epochs, endpoint=False)
fig1 = plt.figure('Metrics')
plt.plot(history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1, max(history.history['val_accuracy']), marker = 'o', markerfacecolor = 'red', markersize = 12)
plt.plot(xdata, Recall, color = 'c', linewidth = 2,linestyle='dashed')
plt.plot(xdata, Specificity, color = 'm',linewidth = 2,linestyle='dashed')
plt.plot(xdata, MCC, color = 'b',linewidth = 2.5)
plt.plot(xdata, F1, color = 'r',linewidth = 2.5)
plt.plot(xdata, history.history['val_auc'], color = 'tab:orange',linewidth = 2.5)
plt.plot(xdata, history.history['val_accuracy'], color = 'tab:olive',linewidth = 2.5)
plt.legend(['Best score','Recall','Specificity','MCC','F1','AUC','Accuracy'])
plt.title('Validation metrics')
plt.xlabel('Epochs')
plt.ylim(0, 1)

In [ ]:
#Show accuracy metrics
fig2 = plt.figure('Accuracy')
plt.plot(history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1, max(history.history['val_accuracy']), marker = 'o', markerfacecolor = 'red', markersize = 12)
plt.plot(xdata, history.history['accuracy'], color = 'tab:green',linewidth = 2.5)
plt.plot(xdata, history.history['val_accuracy'], color = 'tab:olive',linewidth = 2.5)
plt.legend(['Best Score','Accuracy','val_accuracy'])
plt.title('Accuracy vs val_accuracy')
plt.xlabel('Epochs')
plt.ylim(0, 1)

plt.show()

In [ ]:
#Calculating AUC - ROC curve
Y_tf = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y_test, Y_tf)
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
#Show ROC-AUC Curve
plt.figure("ROC-AUC Curve")
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='MobileNetV3 (area = {:.3f})'.format(auc_keras), color = 'tab:orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure('Zoomed')
plt.xlim(0, 0.4)
plt.ylim(0.6, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='MobileNetV3 (area = {:.3f})'.format(auc_keras),color = 'tab:orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')

AUC - ROC curve - Based on: https://medium.com/hackernoon/simple-guide-on-how-to-generate-roc-plot-for-keras-classifier-2ecc6c73115a

In [ ]:
print('=======================================')
print("|            Best Scores               |")
print('=======================================\n')
print('        Epoch :        {}'.format(1 + history.history['val_accuracy'].index(max(history.history['val_accuracy']))))
print('        Val_Accuracy : {:.4f}'.format(history.history['val_accuracy'][history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        Recall :       {:.4f}'.format(Recall[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        Specificity :  {:.4f}'.format(Specificity[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        AUC :          {:.4f}'.format(history.history['val_auc'][history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        F1 Score :     {:.4f}'.format(F1[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        MCC :          {:.4f}'.format(MCC[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]))
print('        Seed:         ',Seed_0)

#print the best result of the whole training process

plt.plot(xdata, Recall, color = 'c', linewidth = 2,linestyle='dashed')
plt.plot(xdata, Specificity, color = 'm',linewidth = 2,linestyle='dashed')
plt.plot(xdata, MCC, color = 'b',linewidth = 2.5)
plt.plot(xdata, F1, color = 'r',linewidth = 2.5)
plt.plot(xdata, history.history['val_auc'], color = 'tab:orange',linewidth = 2.5)
plt.plot(xdata, history.history['val_accuracy'], color = 'tab:olive',linewidth = 2.5)

##Show Validation metrics

plt.figure("Confusion Matrix Visualization")
CM = np.array( [ [TN[history.history['val_accuracy'].index(max(history.history['val_accuracy']))],FP[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]],[FN[history.history['val_accuracy'].index(max(history.history['val_accuracy']))],TP[history.history['val_accuracy'].index(max(history.history['val_accuracy']))]] ] )
sns.heatmap(CM, annot=True)

#Plot CM

#Recover Model

In [ ]:
#Save model 
model.save('Model_1/MobileNetV3.h5')

In [ ]:
#Load the saved model again
new_model = tf.keras.models.load_model( 'Model_1/MobileNetV3.h5', compile=True, options=None )

#Show the model architecture
new_model.summary()

In [ ]:
#Setting Best weights saved

if (history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1) > 9:
    Base = 'training_1/cp-{}.ckpt'
else:
    Base = 'training_1/cp-0{}.ckpt'

BestWeights = Base.format(history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1 )

#Loading Best weights saved on new_model
model.load_weights(BestWeights)

In [ ]:
#Verify metrics
# Evaluate the restored model
loss, acc, aucscore, tp, tn, fp, fn = model.evaluate(X_test, Y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
#Calculating AUC - ROC curve / Restored model
Y_tf = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y_test, Y_tf)
auc_keras = auc(fpr_keras, tpr_keras)

13/13 [==============================] - 6s 463ms/step


In [ ]:
#show ROC-AUC Curve - Recovered model
plt.figure("ROC-AUC Curve - Recovered model")
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='ResNet 50 V2 Restored model (area = {:.3f})'.format(auc_keras), color = 'tab:orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Zoom in view of the upper left corner.
plt.figure('Zoomed - Recovered model')
plt.xlim(0, 0.4)
plt.ylim(0.6, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='ResNet 50 V2 Restored model(area = {:.3f})'.format(auc_keras),color = 'tab:orange')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')

In [ ]:
#Show predicted vs real labels

index = 0
for i in Y_tf:
    if i >= 0.5:
        Y_tf[index] = int(1)
    else:
        Y_tf[index] = int(0)
    index += 1
print("                           *** Real Labels ***\n")
print(Y_test,'\n')
print("                         *** Predicted Labels ***\n")
print( Y_tf)

In [ ]:
#Validate real ACC of restored model

index = 0
tp_val = 0
for c in Y_test:
    if Y_test[index] == Y_tf[index]:
        tp_val += 1
    index += 1
print('  Manual ACC_Val:   {:.2f}%'.format(100*tp_val/len(Y_test)))